In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = ( SparkSession.builder.appName("spark_processing_api")
.config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2")
.getOrCreate() )

spark.sparkContext.setLogLevel("WARN")

22/07/10 11:18:31 WARN Utils: Your hostname, FB1005 resolves to a loopback address: 127.0.1.1; using 192.168.0.71 instead (on interface wlp0s20f3)
22/07/10 11:18:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/renan/.ivy2/cache
The jars for the packages stored in: /home/renan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c3eda77c-d1c4-42be-9900-0ac8e0bf31ef;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 612ms :: artifacts dl 13ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central i

In [3]:
bootstrap_server = '10.108.123.238:9092'

topics = [
'ingestion_src_api_iq_BTCUSD_1_json', 'ingestion_src_api_iq_BTCUSD_5_json', 'ingestion_src_api_iq_BTCUSD_60_json', 'ingestion_src_api_iq_BTCUSD_300_json', 'ingestion_src_api_iq_BTCUSD_900_json', 'ingestion_src_api_iq_BTCUSD_1800_json', 'ingestion_src_api_iq_BTCUSD_3600_json', 'ingestion_src_api_iq_BTCUSD_14400_json', 'ingestion_src_api_iq_BTCUSD_86400_json',
'ingestion_src_api_iq_XRPUSD_1_json', 'ingestion_src_api_iq_XRPUSD_5_json', 'ingestion_src_api_iq_XRPUSD_60_json', 'ingestion_src_api_iq_XRPUSD_300_json', 'ingestion_src_api_iq_XRPUSD_900_json', 'ingestion_src_api_iq_XRPUSD_1800_json', 'ingestion_src_api_iq_XRPUSD_3600_json', 'ingestion_src_api_iq_XRPUSD_14400_json', 'ingestion_src_api_iq_XRPUSD_86400_json',
'ingestion_src_api_iq_ETHUSD_1_json', 'ingestion_src_api_iq_ETHUSD_5_json', 'ingestion_src_api_iq_ETHUSD_60_json', 'ingestion_src_api_iq_ETHUSD_300_json', 'ingestion_src_api_iq_ETHUSD_900_json', 'ingestion_src_api_iq_ETHUSD_1800_json', 'ingestion_src_api_iq_ETHUSD_3600_json', 'ingestion_src_api_iq_ETHUSD_14400_json', 'ingestion_src_api_iq_ETHUSD_86400_json',
'ingestion_src_api_iq_OMGUSD_1_json', 'ingestion_src_api_iq_OMGUSD_5_json', 'ingestion_src_api_iq_OMGUSD_60_json', 'ingestion_src_api_iq_OMGUSD_300_json', 'ingestion_src_api_iq_OMGUSD_900_json', 'ingestion_src_api_iq_OMGUSD_1800_json', 'ingestion_src_api_iq_OMGUSD_3600_json', 'ingestion_src_api_iq_OMGUSD_14400_json', 'ingestion_src_api_iq_OMGUSD_86400_json',
'ingestion_src_api_iq_TRXUSD_1_json', 'ingestion_src_api_iq_TRXUSD_5_json', 'ingestion_src_api_iq_TRXUSD_60_json', 'ingestion_src_api_iq_TRXUSD_300_json', 'ingestion_src_api_iq_TRXUSD_900_json', 'ingestion_src_api_iq_TRXUSD_1800_json', 'ingestion_src_api_iq_TRXUSD_3600_json', 'ingestion_src_api_iq_TRXUSD_14400_json', 'ingestion_src_api_iq_TRXUSD_86400_json',
]

schema = "active_id INT, size INT, at STRING, from STRING, to STRING, id INT, open FLOAT, close FLOAT, min FLOAT, max FLOAT, ask FLOAT, bid FLOAT, volume FLOAT, phase STRING"

df = (spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", bootstrap_server)
    .option("subscribePattern", "ingestion_src_api.*")
    .option("startingOffsets", "latest")
    .load()
    .select(col("topic").cast("string"), from_json(col("value").cast("string"), schema).alias("value")) 
    .withColumn("topic", regexp_replace("topic", "ingestion", "processed"))
    .select("topic", "value.active_id", "value.size", "value.id", "value.open")
    # .select(to_json(struct(expr("value.active_id as active_id"), expr("value.size as timeframe"),
    # expr("cast(value.at / 1000000000 as timestamp) as executed_at"), expr("FROM_UNIXTIME(value.from) as candle_from"), 
    # expr("FROM_UNIXTIME(value.to) as candle_to"), expr("value.id as period"), 
    # "value.open", "value.close", "value.min", "value.max", "value.ask", "value.bid", "value.volume")).alias("value"))
    .writeStream.format("console").option("truncate", False).start().awaitTermination() 
    )
#for topic in topics:
# df = (df.select(to_json(struct(expr("value.active_id as active_id"), expr("value.size as timeframe"),
#         expr("cast(value.at / 1000000000 as timestamp) as executed_at"), expr("FROM_UNIXTIME(value.from) as candle_from"), 
#         expr("FROM_UNIXTIME(value.to) as candle_to"), expr("value.id as period"), 
#         "value.open", "value.close", "value.min", "value.max", "value.ask", "value.bid", "value.volume")).alias("value"))
#         #.where("topic = '{}'".format(topic))
#         #.writeStream.format("kafka").option("kafka.bootstrap.servers", bootstrap_server)
#         #.option("topic", df.select("topic")topic.replace('ingestion', 'processed'))
#         .option("checkpointLocation", f"./checkpoint_/")#{topic.replace('ingestion', 'processed')}
#         .start()
#         )

#

22/07/10 11:18:38 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-72c69a31-f308-47db-aac0-f13ac401d561. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+----------------------------------+---------+----+---------+-------+
|topic                             |active_id|size|id       |open   |
+----------------------------------+---------+----+---------+-------+
|processed_src_api_iq_OMGUSD_1_json|825      |1   |147354536|1.85905|
+----------------------------------+---------+----+---------+-------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----------------------------------+---------+----+---------+---------+
|topic                             |active_id|size|id       |open     |
+----------------------------------+---------+----+---------+---------+
|processed_src_api_iq_TRXUSD_1_json|858      |1   |140240446|0.068565 |
|processed_src_api_iq_TRXUSD_1_json|858      |1   |140240445|0.068565 |
|processed_src_api_iq_TRXUSD_1_json|858      |1   |140240447|0.068565 |
|processed_src_api_iq_TRXUSD_1_json|858      |1   |140240448|0.068575 |
|processed_src_api_iq_XRPUSD_1_json|817      |1   |148084740|0.330995 |
|processed_src_api_iq_XRPUSD_1_json|817      |1   |148084741|0.33078  |
|processed_src_api_iq_ETHUSD_1_json|818      |1   |148084740|1170.214 |
|processed_src_api_iq_ETHUSD_1_json|818      |1   |148084742|1170.752 |
|processed_src_api_iq_BTCUSD_1_json|816      |1   |148084741|21020.621|
|processed_src_api_iq_OMGUSD_1_json|825

-------------------------------------------
Batch: 2
-------------------------------------------
+----------------------------------+---------+----+---------+---------+
|topic                             |active_id|size|id       |open     |
+----------------------------------+---------+----+---------+---------+
|processed_src_api_iq_TRXUSD_1_json|858      |1   |140240449|0.068585 |
|processed_src_api_iq_XRPUSD_1_json|817      |1   |148084744|0.330945 |
|processed_src_api_iq_ETHUSD_5_json|818      |5   |29616949 |1170.241 |
|processed_src_api_iq_ETHUSD_1_json|818      |1   |148084745|1170.7075|
|processed_src_api_iq_BTCUSD_5_json|816      |5   |29616949 |21020.615|
|processed_src_api_iq_XRPUSD_1_json|817      |1   |148084743|0.331075 |
|processed_src_api_iq_OMGUSD_1_json|825      |1   |147354540|1.86     |
|processed_src_api_iq_ETHUSD_1_json|818      |1   |148084744|1170.241 |
|processed_src_api_iq_BTCUSD_1_json|816      |1   |148084744|21021.973|
|processed_src_api_iq_OMGUSD_1_json|825

-------------------------------------------
Batch: 3
-------------------------------------------
+----------------------------------+---------+----+---------+---------+
|topic                             |active_id|size|id       |open     |
+----------------------------------+---------+----+---------+---------+
|processed_src_api_iq_TRXUSD_1_json|858      |1   |140240450|0.068585 |
|processed_src_api_iq_OMGUSD_1_json|825      |1   |147354542|1.86     |
|processed_src_api_iq_TRXUSD_1_json|858      |1   |140240451|0.068585 |
|processed_src_api_iq_XRPUSD_1_json|817      |1   |148084745|0.33097  |
|processed_src_api_iq_XRPUSD_1_json|817      |1   |148084746|0.330935 |
|processed_src_api_iq_ETHUSD_1_json|818      |1   |148084746|1171.3295|
|processed_src_api_iq_BTCUSD_1_json|816      |1   |148084745|21025.434|
+----------------------------------+---------+----+---------+---------+



...

-------------------------------------------
Batch: 5
-------------------------------------------
+----------------------------------+---------+----+---------+---------+
|topic                             |active_id|size|id       |open     |
+----------------------------------+---------+----+---------+---------+
|processed_src_api_iq_TRXUSD_1_json|858      |1   |140240453|0.068585 |
|processed_src_api_iq_OMGUSD_5_json|825      |5   |29470910 |1.861    |
|processed_src_api_iq_BTCUSD_5_json|816      |5   |29616950 |21031.486|
|processed_src_api_iq_BTCUSD_1_json|816      |1   |148084748|21031.486|
|processed_src_api_iq_ETHUSD_1_json|818      |1   |148084748|1170.7465|
|processed_src_api_iq_BTCUSD_1_json|816      |1   |148084747|21030.783|
+----------------------------------+---------+----+---------+---------+



-------------------------------------------
Batch: 6
-------------------------------------------
+----------------------------------+---------+----+---------+---------+
|topic                             |active_id|size|id       |open     |
+----------------------------------+---------+----+---------+---------+
|processed_src_api_iq_TRXUSD_1_json|858      |1   |140240454|0.068585 |
|processed_src_api_iq_XRPUSD_5_json|817      |5   |29616950 |0.33115  |
|processed_src_api_iq_XRPUSD_1_json|817      |1   |148084749|0.331065 |
|processed_src_api_iq_ETHUSD_1_json|818      |1   |148084749|1170.6235|
|processed_src_api_iq_ETHUSD_5_json|818      |5   |29616950 |1170.6235|
|processed_src_api_iq_OMGUSD_1_json|825      |1   |147354545|1.861    |
|processed_src_api_iq_XRPUSD_1_json|817      |1   |148084748|0.33115  |
|processed_src_api_iq_TRXUSD_5_json|858      |5   |28048091 |0.068585 |
+----------------------------------+---------+----+---------+---------+



-------------------------------------------
Batch: 7
-------------------------------------------
+----------------------------------+---------+----+---------+---------+
|topic                             |active_id|size|id       |open     |
+----------------------------------+---------+----+---------+---------+
|processed_src_api_iq_ETHUSD_1_json|818      |1   |148084750|1170.535 |
|processed_src_api_iq_OMGUSD_1_json|825      |1   |147354546|1.861    |
|processed_src_api_iq_BTCUSD_1_json|816      |1   |148084749|21032.613|
|processed_src_api_iq_OMGUSD_1_json|825      |1   |147354547|1.861    |
+----------------------------------+---------+----+---------+---------+

-------------------------------------------
Batch: 8
-------------------------------------------
+----------------------------------+---------+----+---------+---------+
|topic                             |active_id|size|id       |open     |
+----------------------------------+---------+----+---------+---------+
|processed_sr

-------------------------------------------
Batch: 9
-------------------------------------------
+----------------------------------+---------+----+---------+--------+
|topic                             |active_id|size|id       |open    |
+----------------------------------+---------+----+---------+--------+
|processed_src_api_iq_TRXUSD_1_json|858      |1   |140240457|0.068595|
|processed_src_api_iq_XRPUSD_1_json|817      |1   |148084751|0.33136 |
|processed_src_api_iq_OMGUSD_1_json|825      |1   |147354548|1.861   |
|processed_src_api_iq_ETHUSD_1_json|818      |1   |148084752|1171.12 |
+----------------------------------+---------+----+---------+--------+

-------------------------------------------
Batch: 10
-------------------------------------------
+----------------------------------+---------+----+---------+---------+
|topic                             |active_id|size|id       |open     |
+----------------------------------+---------+----+---------+---------+
|processed_src_api_i

KeyboardInterrupt: 